<a href="https://colab.research.google.com/github/bm627/Masters-Sweepstake/blob/main/Masters_Sweepstake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from datetime import datetime, timedelta

from google.colab import auth
auth.authenticate_user()

from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Use the name of your Google Sheet here instead of 'Elastic_Beams_Template'. 
spreadsheet = gc.open('Masters Sweepstake Live')

# Load the worksheet.
worksheet = spreadsheet.worksheet('Entry')

# Get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame.
df = pd.DataFrame.from_records(rows)

CUGCandPlayer = df.to_dict()

def getData(day):
  url = "https://www.golfchannel.com/api/v2/events/19956/leaderboard"

  payload = ""
  response = requests.request("GET", url, data=payload)

  jsondata = json.loads(response.text)

  playersAndScores = {}
  names = []
  todayScores = []
  leaderboardScores = []

  #Running through all imported data to find players and their leaderboard scores
  for i in range(len(jsondata['result']['golfers'])):

      #Getting first and last names, added together later to create full name
      firstName = jsondata['result']['golfers'][i]['firstName']
      lastName = jsondata['result']['golfers'][i]['lastName']
      name = firstName + ' ' + lastName
      #Getting today's score  
      scoresToPar = []  

      #Trying to get round 1 score
      try:
          round1ScoreDict = jsondata['result']['golfers'][i]['leaderboardRounds'][0]
          round1Score = round1ScoreDict.get('roundScore')
          scoresToPar.append(round1Score - 72)
      #If not, take live score
      except:
          livescore = jsondata['result']['golfers'][i]['todayPar'] 
          if livescore == None:
            scoresToPar.append(0)
          elif livescore != None:
            scoresToPar.append(livescore) 

      #Trying to get round 2 complete score
      try:
          round2ScoreDict = jsondata['result']['golfers'][i]['leaderboardRounds'][1]
          round2Score = round2ScoreDict.get('roundScore')
          scoresToPar.append(round2Score - 72)
      #If not, take live score
      except:
          livescore = jsondata['result']['golfers'][i]['todayPar'] 
          if livescore == None:
            scoresToPar.append(0)
          elif livescore != None:
            if day == 2:
              scoresToPar.append(livescore)
            else:
              scoresToPar.append(0)

      #Trying to get round 3 complete score
      try:
          round3ScoreDict = jsondata['result']['golfers'][i]['leaderboardRounds'][2]
          round3Score = round3ScoreDict.get('roundScore')
          scoresToPar.append(round3Score - 72)
      #If not, take live score
      except:
          livescore = jsondata['result']['golfers'][i]['todayPar'] 
          if livescore == None:
            scoresToPar.append(0)
          elif livescore != None:
            if day == 3:
              scoresToPar.append(livescore)
            else:
              scoresToPar.append(0)

      #Trying to get round 4 complete score
      try:
          round4ScoreDict = jsondata['result']['golfers'][i]['leaderboardRounds'][3]
          round4Score = round4ScoreDict.get('roundScore')
          scoresToPar.append(round4Score - 72)
      #If not, take live score
      except:
          livescore = jsondata['result']['golfers'][i]['todayPar'] 
          if livescore == None:
            scoresToPar.append(0)
          elif livescore != None:
            if day == 4:
              scoresToPar.append(livescore)
            else:
              scoresToPar.append(0)
      #Creating Dictionary
      newDict = {name:scoresToPar}
      playersAndScores.update(newDict)

  return playersAndScores

def uploadPlayersLeaderboard(playersAndScores):
  #Creating the list of people
  players = []
  for person in playersAndScores.keys():
      players.append(person)
  
  #Creating the corresponding list of scores
  R1 = []
  R2 = []
  R3 = []
  R4 = []
  for score in playersAndScores.values():
    R1.append(score[0])
    R2.append(score[1])
    R3.append(score[2])
    R4.append(score[3])

  #Creating time updated list
  now = datetime.now()+ timedelta(hours = 1)
  current_time = now.strftime("%H:%M:%S") 

  timeList = [current_time] + [0] * (len(playersAndScores.keys())-1)

  # Create empty dataframe
  df = pd.DataFrame()

  # Create a column
  df['Players'] = players
  df['R1'] = R1
  df['R2'] = R2
  df['R3'] = R3
  df['R4'] = R4
  df['Time Updated'] = timeList
  
  # Use the name of your Google Sheet here instead of 'Elastic_Beams_Template'. 
  spreadsheet = gc.open('Masters Sweepstake Live')

  # Load the worksheet.
  worksheet = spreadsheet.worksheet('Player Scores')
  set_with_dataframe(worksheet, df) 

def create7List(CUGCandPlayer):
  listOf7List = []
  for i in range(len(CUGCandPlayer)):
    playerAndChoices = CUGCandPlayer.get(i)
    CUGCandChoices = []
    for i in range(len(playerAndChoices)):
      CUGCandChoices.append(playerAndChoices.get(i))
    listOf7List.append(CUGCandChoices)
  return(listOf7List)

def createPersonAndChoiceDict():
    sevenList = create7List(CUGCandPlayer)
    personAndChoiceDict = {}
    for i in range(len(sevenList)):
      person = sevenList[i][0]
      choices = sevenList[i][1:7]
      newDict = {person:choices}
      personAndChoiceDict.update(newDict)
    return(personAndChoiceDict)
  
#Converting list of 6 string scores to integer scores, sorting, taking top 4 and adding to give total CUGC score for the day
def scoresFrom6(scores):
    #Creating empty list to score int scores in
    scoreInts = []

    #Iterating through, converting string scores to ints and adding to list
    for score in scores:
      scoreInts.append(int(score))
    
    #Sorting to find top 4
    scoreInts.sort()
    topFour = scoreInts[0:4]

    #Finding and returning the total
    total = sum(topFour)
    return total

def calculateCUGCScores(playersAndScores):

  #Dictionary of CUGC
  personAndChoiceDict = createPersonAndChoiceDict()

  #Empty list of people, to be added to
  peopleAndScores = {}

  #Iterating through CUGC 
  for person in personAndChoiceDict:

    #Finding chosen 6 players, output their scores, over 4 days
    chosenPlayers = personAndChoiceDict.get(person)
    scores = []
    for i in range(4):
      for player in chosenPlayers:
        if player in playersAndScores:
          dayScore = playersAndScores.get(player)
          scores.append(dayScore[i])
        else:
          pass
    #Selecting the days, by taking chunks of 6
    day1 = scores[0:6]
    day2 = scores[6:12]
    day3 = scores[12:18]
    day4 = scores[18:24]

    #Calculating the 'score' for each CUGC each day
    day1Top4Score = scoresFrom6(day1)
    day2Top4Score = scoresFrom6(day2)
    day3Top4Score = scoresFrom6(day3)
    day4Top4Score = scoresFrom6(day4)
    total = day1Top4Score + day2Top4Score + day3Top4Score + day4Top4Score
    scoresList = [day1Top4Score, day2Top4Score, day3Top4Score, day4Top4Score, total]
    newDict = {person:scoresList}
    peopleAndScores.update(newDict)
  sortedPeopleAndScores = dict(sorted(peopleAndScores.items(), key=lambda x:x[1][4]))
  return sortedPeopleAndScores

def uploadScores(data):

  #Creating time updated list
  now = datetime.now()+ timedelta(hours = 1)
  current_time = now.strftime("%H:%M:%S") 
  print('Updated at: ' + current_time)

  #Creating the list of people
  people = []
  for person in data.keys():
      people.append(person)

  #Creating list of numerical data
  R1 = []
  R2 = []
  R3 = []
  R4 = []
  Total = []
  
  for numbers in data.values():
    R1.append(numbers[0])
    R2.append(numbers[1])
    R3.append(numbers[2])
    R4.append(numbers[3])
    Total.append(numbers[4])

  # Create empty dataframe
  df = pd.DataFrame()

  # Create a column
  df['CUGC'] = people
  df['Day 1'] = R1
  df['Day 2'] = R2
  df['Day 3'] = R3
  df['Day 4'] = R4
  df['Total'] = Total
  
  # Use the name of your Google Sheet here instead of 'Elastic_Beams_Template'. 
  spreadsheet = gc.open('Masters Sweepstake Live')

  # Load the worksheet.
  worksheet = spreadsheet.worksheet('Leaderboard')
  set_with_dataframe(worksheet, df) 

if __name__ == "__main__":
  while True:
    playersAndScores = getData(2)
    uploadPlayersLeaderboard(playersAndScores)
    sortedPeopleAndScores = calculateCUGCScores(playersAndScores)
    uploadScores(sortedPeopleAndScores)
    time.sleep(60)

Updated at: 20:34:44
Updated at: 20:35:45
Updated at: 20:36:47
Updated at: 20:37:48
Updated at: 20:38:49
Updated at: 20:39:50
Updated at: 20:40:51
Updated at: 20:41:52
